In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from NN_Classification import NeuralNetworkClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,7900,1166,D-penicillamine,16839,F,N,N,N,N,0.8,309.0,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C
7901,7901,1492,Placebo,17031,F,N,Y,N,N,0.9,260.0,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C
7902,7902,1576,D-penicillamine,25873,F,N,N,Y,S,2.0,225.0,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D
7903,7903,3584,D-penicillamine,22960,M,N,Y,N,N,0.7,248.0,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D


In [3]:
X = df.drop('Status', axis=1)
y = df['Status']

In [4]:
encoder = LabelEncoder()

for col in X.columns:
    if X[col].dtype == 'object':
        print(col, X[col].unique())
        X[col] = encoder.fit_transform(X[col])

for col in test.columns:
    if test[col].dtype == 'object':
        test[col] = encoder.fit_transform(test[col])

df

Drug ['D-penicillamine' 'Placebo']
Sex ['M' 'F']
Ascites ['N' 'Y']
Hepatomegaly ['N' 'Y']
Spiders ['N' 'Y']
Edema ['N' 'Y' 'S']


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,7900,1166,D-penicillamine,16839,F,N,N,N,N,0.8,309.0,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C
7901,7901,1492,Placebo,17031,F,N,Y,N,N,0.9,260.0,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C
7902,7902,1576,D-penicillamine,25873,F,N,N,Y,S,2.0,225.0,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D
7903,7903,3584,D-penicillamine,22960,M,N,Y,N,N,0.7,248.0,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D


In [5]:
y_encoder = OneHotEncoder(sparse_output=False)

y = y.to_numpy().reshape(-1, 1)
y = y_encoder.fit_transform(y)

In [6]:
z_scores = np.abs(zscore(X))
threshold = 3
mask = (z_scores < threshold).all(axis=1)
X = X[mask]
y = y[mask]
X.shape, y.shape

((6148, 19), (6148, 3))

In [7]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [8]:
test_id = test['id']
test_id

0        7905
1        7906
2        7907
3        7908
4        7909
        ...  
5266    13171
5267    13172
5268    13173
5269    13174
5270    13175
Name: id, Length: 5271, dtype: int64

In [9]:
scaler = StandardScaler()

df_data = scaler.fit_transform(X)
df = pd.DataFrame(
    data=df_data,
    columns=df.drop('Status', axis=1).columns
)

test_data = scaler.transform(test)
test = pd.DataFrame(
    data=test_data,
    columns=test.columns
)

df = df.drop('id', axis=1)
test = test.drop('id', axis=1)

X = df.to_numpy()
test = test.to_numpy()

df

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,0.381118,0.964834,0.324878,0.0,0.0,-0.898428,-0.502896,-0.206310,-0.408901,0.379173,-0.172059,-0.079479,-0.055410,0.650522,-0.493219,1.127253,0.782241,0.057895
1,0.383043,0.964834,0.107084,0.0,0.0,-0.898428,-0.502896,-0.206310,-0.566646,-0.585683,-0.304601,-0.182125,0.113212,-0.878359,-0.296084,0.005028,0.318478,0.057895
2,-1.337671,0.964834,-0.398018,0.0,0.0,1.113056,-0.502896,-0.206310,-0.303738,0.218363,0.192432,-0.079479,-0.260449,0.426783,-0.296084,0.358773,0.163891,1.217298
3,-1.419472,-1.036448,0.334128,0.0,0.0,1.113056,-0.502896,-0.206310,-0.566646,-0.844765,-0.437143,-0.674828,3.915696,-1.135057,-0.986058,-0.678066,1.555178,0.057895
4,-0.844939,0.964834,-0.105103,0.0,0.0,-0.898428,-0.502896,-0.206310,-0.356320,0.057554,-0.801634,-0.490064,0.132212,0.725102,-0.443935,0.273386,-1.072809,0.057895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6143,-0.736191,0.964834,1.834297,0.0,0.0,-0.898428,-0.502896,-0.206310,-0.198575,-0.532080,0.457516,-0.038421,0.423542,-0.430882,-0.739639,-1.068405,0.782241,0.057895
6144,-0.973896,-1.036448,-0.347283,0.0,0.0,-0.898428,-0.502896,-0.206310,-0.461483,-0.112189,-0.105788,-0.592711,0.094213,-0.691910,2.858083,0.919885,-0.918221,-1.101509
6145,-0.660164,0.964834,-0.293465,0.0,0.0,1.113056,-0.502896,-0.206310,-0.408901,-0.549948,-0.536550,-0.100008,-0.055410,0.822536,-0.739639,0.102612,-0.763634,1.217298
6146,-0.579325,-1.036448,2.184953,0.0,0.0,-0.898428,1.988484,4.847063,0.169496,-0.862633,-1.331803,-0.325830,-0.456780,-0.915649,-1.133909,-0.836641,3.410228,-1.101509


In [10]:
X

array([[ 0.38111847,  0.96483394,  0.32487787, ...,  1.12725287,
         0.78224072,  0.05789473],
       [ 0.3830432 ,  0.96483394,  0.10708422, ...,  0.00502765,
         0.31847834,  0.05789473],
       [-1.33767112,  0.96483394, -0.39801766, ...,  0.35877256,
         0.16389087,  1.21729799],
       ...,
       [-0.66016392,  0.96483394, -0.2934655 , ...,  0.10261245,
        -0.76363391,  1.21729799],
       [-0.57932499, -1.03644779,  2.18495337, ..., -0.83664127,
         3.4102276 , -1.10150853],
       [-0.19245298, -1.03644779,  0.32487787, ...,  0.82230037,
        -0.29987152,  0.05789473]])

In [11]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [12]:
y_encoder.categories_

[array(['C', 'CL', 'D'], dtype=object)]

In [13]:
model = NeuralNetworkClassifier(
    layer_dims=[X.shape[1], 64, 32, 16, 8, 8, y.shape[1]],
    activations=['leaky_relu'] * 5 + ['softmax'],
    epoch=300,
    batch_size=32,
    dropout_prob=0.1,
    learning_rate=0.1,
    early_stopping=True,
    verbose=1,
    patience=5
)

Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [14]:
model.fit(X, y)

Using GPU
GPU Device Name: NVIDIA GeForce RTX 4060 Laptop GPU
Epoch 1/300 - Loss: 0.7284 - Acc: 0.7960 - LR: 0.099593 - Time: 1.15s
Epoch 2/300 - Loss: 0.5572 - Acc: 0.7759 - LR: 0.099187 - Time: 0.90s
Epoch 3/300 - Loss: 0.5988 - Acc: 0.7389 - LR: 0.098783 - Time: 0.93s
Epoch 4/300 - Loss: 0.5262 - Acc: 0.8118 - LR: 0.098380 - Time: 0.96s
Epoch 5/300 - Loss: 0.5581 - Acc: 0.7957 - LR: 0.097980 - Time: 0.92s
Epoch 6/300 - Loss: 0.5409 - Acc: 0.8110 - LR: 0.097580 - Time: 0.94s
Epoch 7/300 - Loss: 0.5130 - Acc: 0.8076 - LR: 0.097183 - Time: 0.93s
Epoch 8/300 - Loss: 0.5081 - Acc: 0.8128 - LR: 0.096787 - Time: 0.92s
Epoch 9/300 - Loss: 0.5087 - Acc: 0.8134 - LR: 0.096393 - Time: 0.95s
Epoch 10/300 - Loss: 0.5254 - Acc: 0.8170 - LR: 0.096000 - Time: 1.07s
Epoch 11/300 - Loss: 0.5446 - Acc: 0.7806 - LR: 0.095609 - Time: 1.00s
Epoch 12/300 - Loss: 0.5011 - Acc: 0.8292 - LR: 0.095219 - Time: 0.94s
Epoch 13/300 - Loss: 0.5320 - Acc: 0.7838 - LR: 0.094831 - Time: 0.92s
Epoch 14/300 - Loss: 0.4

In [15]:
probs = model.predict_proba(test)
probs

array([[0.81043757, 0.01856715, 0.17099528],
       [0.9524735 , 0.00689018, 0.04063632],
       [0.24546138, 0.0460109 , 0.70852773],
       ...,
       [0.7239147 , 0.02455131, 0.25153399],
       [0.97420522, 0.00439946, 0.02139532],
       [0.46051682, 0.03925374, 0.50022943]])

In [16]:
y_pred = model.predict(X)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
accuracy_score(y_pred=y_pred, y_true=y.argmax(axis=1))

0.8319778789850358

In [18]:
probs_df = pd.DataFrame(probs.reshape(-1, 3), columns=[f'Status_{i}' for i in y_encoder.categories_[0]])
probs_df

,Status_C,Status_CL,Status_D
0,0.810438,0.018567,0.170995
1,0.952473,0.006890,0.040636
2,0.245461,0.046011,0.708528
3,0.966471,0.005346,0.028182
4,0.724277,0.024527,0.251196
...,...,...,...
5266,0.965173,0.005499,0.029328
5267,0.967173,0.005263,0.027564
5268,0.723915,0.024551,0.251534
5269,0.974205,0.004399,0.021395


In [19]:
probs_df.insert(0, 'id', test_id.values)
probs_df

,id,Status_C,Status_CL,Status_D
0,7905,0.810438,0.018567,0.170995
1,7906,0.952473,0.006890,0.040636
2,7907,0.245461,0.046011,0.708528
3,7908,0.966471,0.005346,0.028182
4,7909,0.724277,0.024527,0.251196
...,...,...,...,...
5266,13171,0.965173,0.005499,0.029328
5267,13172,0.967173,0.005263,0.027564
5268,13173,0.723915,0.024551,0.251534
5269,13174,0.974205,0.004399,0.021395


In [20]:
probs_df.to_csv('predicted_probabilities_mymodel_2.csv', index=False)